# Lab 01 - Parte 1: Índice Invertido e Busca Booleana

Importando as bibliotecas necessárias:

In [1]:
import pandas as pd
import numpy as np
import nltk as nl
from collections import defaultdict

Conhecendo os dados:

In [2]:
df = pd.read_csv('noticias_estadao.csv')
df.head()

titulo  \
0   11 dos eleitores do País são filiados a legendas   
1  11 executivos integram 1º pedido de condenação...   
2  11 executivos integram 1º pedido de condenação...   
3         13 de deputados do PMDB quer romper com PT   
4                               2014 começou em 2007   

                                            conteudo  idNoticia  
0  Há porém variações regionais nesse fenômeno En...       7617  
1  CURITIBA A força-tarefa da Operação Lava Jato ...        412  
2  CURITIBA A força-tarefa da Operação Lava Jato ...        415  
3  O Estado ouviu 54 dos 74 deputados do PMDB em ...       6736  
4  O estudo do Estadão Dados publicado ontem sobr...       7611

Agora, é preciso tratar os dados do csv em questão para podermos usá-los na resolução do problema, transformando cada palavra do documento em tokens e colocando-os em um dicionário. Para isso, utilizaremos a função word_tokenize() da biblioteca nltk. Para que não haja problemas com diferenciação de letras maíusculas e minúsculas na hora da pesquisa (Marcela != marcela), transformamos todas as letras da pesquisa em letras minúsculas.  

In [3]:
def make_tokens(df):
    inverted_index = defaultdict(set)
    for i, row in df.iterrows():
        tokens_titulos = (word.lower() for word in (nl.word_tokenize(row['titulo'])))
        
        for token in tokens_titulos:
            inverted_index[token].add(row['idNoticia'])
            
        tokens_conteudo = (word.lower() for word in (nl.word_tokenize(row['conteudo'])))
        for token in tokens_conteudo:
            inverted_index[token].add(row['idNoticia'])
        
    return inverted_index

dicionario = make_tokens(df)

Foi pedido para que se implementasse 3 algoritmos:
- 1-termo: usuário passa um termo e o algoritmo retorna os documentos associados;
- Busca AND: todos os termos da consulta devem estar presentes na página;
- Busca OR: algum dos termos da consulta deve estar presente na página.


In [10]:
def termo(termo):
    return dicionario[termo]

#busca por termos AND
def busca_and(termo1, termo2):
    busca1 = dicionario[termo1]
    busca2 = dicionario[termo2]
    return busca1 & busca2

#busca por temos OR
def busca_or(termo1, termo2):
    busca1 = dicionario[termo1]
    busca2 = dicionario[termo2]
    return busca1 | busca2

Método de busca final, que recebe uma string de pesquisa (que é um array de caracteres em python) e retorna o array resultante com os ids das notícias correspondentes à pesquisa:

In [11]:
def busca(pesquisa):
    pesquisa = pesquisa.split()
    if len(pesquisa) == 1:
        return termo(pesquisa[0].lower())
    else:
        if pesquisa[1] == "AND":
            return busca_and(pesquisa[0].lower(), pesquisa[2].lower())
        elif pesquisa[1] == "OR":
            return busca_or(pesquisa[0].lower(), pesquisa[2].lower())
        else:
            return "Não encontrado!"

In [12]:
assert len(busca("debate OR presidencial"))== 1770
assert len(busca("debate AND presidencial")) == 201
assert len(busca("presidenciáveis OR corruptos")) == 164
assert len(busca("presidenciáveis AND corruptos"))== 0
assert len(busca("Belo OR Horizonte")) == 331
assert len(busca("Belo AND Horizonte")) == 242